In [1]:
import pandas as pd
import pickle
from utils import *

config = load_config()

In [2]:
PROJECT_PATH = config.project_path
DATA_PATH = PROJECT_PATH.joinpath("data/processed")

In [3]:
with open(DATA_PATH.joinpath("gpt4o_mini_generated_synthetic_conversation_2.pkl"), "rb") as f :
    data = pickle.load(f)

In [ ]:
print(data[0]['messages'])

In [5]:
from datasets import Dataset

def form_dataset() :
    with open(DATA_PATH.joinpath("synthetic_notes_for_sft.pkl"), "rb") as f :
        notes = pickle.load(f)
    
    with open(DATA_PATH.joinpath("synthetic_conversation_10000.pkl"), "rb") as f :
        chat = pickle.load(f)
        # sft_chat = []
        # for i in notes.index :
        #     sft_chat.append(chat[i])
        # chat = sft_chat

    prompt = config.load_prompts("autoeval_stage1.txt")
    
    new_rows = []
    for i, row in notes.iterrows() :
    #     # q = str(q)
        note = row['note']
        conversation = chat[i]
        system_prompt = prompt.format(discharge_note=note)

        new_rows.append({"messages" : [{"role" : "system", "content" : system_prompt}] + conversation })
    new_rows = Dataset.from_list(new_rows)
    return new_rows


In [6]:
def load_datasets() :
    df = form_dataset()
    # df = Dataset.from_list(df)
    df = df.train_test_split(train_size=0.8, seed=0)
    return df

In [7]:
df = load_datasets()

token id for EOC token

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModelForCausalLM

MODEL_PATH = "/home/wjang/2024_chatbot_noteaid/model/gguf/llama3.2-3B-lora"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
# model = PeftModelForCausalLM.from_pretrained(MODEL_PATH)

In [15]:
tokenizer.convert_tokens_to_ids("<EOC>")

128256